<a href="https://colab.research.google.com/github/Risskr/Stock-App/blob/Working/StocksApp_DashApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Section 1: Set Up**


In [2]:
# #-----refesh python script-------#
# get_ipython().kernel.do_shutdown(restart=True)

# #----GPU Setup------#
# #Check GPUs
# !nvidia-smi

# #load cudf to use GPUs for analysis
# %load_ext cudf.pandas


#------Imports--------#
import pandas as pd
import datetime
import pickle
import numpy as np
import requests
import time
import os


#-------Froms-------#
from tqdm.notebook import tqdm
from datetime import datetime, timedelta


# #------Force Connect MyDrive-------#
# from google.colab import drive
# drive.mount('/content/drive')


# **Section 2: Stock Data**

## Ensure the last 6 months of EODHD Data is available

In [3]:
# Ensure the last 6 months of EODHD Data is available
"""
Returns:
nasdaq_df:
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 627225 entries, 0 to 627224
  Data columns (total 8 columns):
  #   Column          Non-Null Count   Dtype
  ---  ------          --------------   -----
  0   date            627225 non-null  datetime64[ns]
  1   ticker          627098 non-null  object
  2   open            627225 non-null  float64
  3   high            627225 non-null  float64
  4   low             627225 non-null  float64
  5   close           627225 non-null  float64
  6   adjusted_close  627225 non-null  float64
  7   volume          627225 non-null  float64
"""
#API_KEY = 'demo'  # Replace with your API key if not using demo
API_KEY = '68433aff09ea73.10710364'
EXCHANGE = 'NASDAQ'
DAYS_BACK = 180  # Approx. 6 months
SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv'
MAX_CALLS_PER_RUN = 200  # Use 1–5 for the free tier
SECONDS_BETWEEN_CALLS = 0  # Add delay to be respectful

# Generate past 6 months of weekdays
today = datetime.utcnow().date()
dates = [today - timedelta(days=i) for i in range(DAYS_BACK)]
dates = sorted([d for d in dates if d.weekday() < 5])  # Keep only weekdays

# Load already-downloaded dates if file exists
downloaded_dates = set()
if os.path.exists(SAVE_PATH):
    df_existing = pd.read_csv(SAVE_PATH)
    downloaded_dates = set(pd.to_datetime(df_existing['date']).dt.date)
# Filter to only dates we haven’t downloaded
pending_dates = [d for d in dates if d not in downloaded_dates]

# Prepare data holder
all_data = []

for i, date in enumerate(pending_dates[:MAX_CALLS_PER_RUN]):
    date_str = date.strftime('%Y-%m-%d')
    url = f'https://eodhd.com/api/eod-bulk-last-day/{EXCHANGE}?api_token={API_KEY}&fmt=json&date={date_str}'
    print(f"[{i+1}] Fetching {date_str}...")

    try:
        response = requests.get(url)
        response.raise_for_status()
        day_data = response.json()

        for entry in day_data:
            all_data.append({
                'date': entry.get('date'),
                'ticker': entry.get('code'),
                'open': entry.get('open'),
                'high': entry.get('high'),
                'low': entry.get('low'),
                'close': entry.get('close'),
                'adjusted_close': entry.get('adjusted_close'),
                'volume': entry.get('volume'),
            })

        # Respect the delay
        if i < MAX_CALLS_PER_RUN - 1:
            time.sleep(SECONDS_BETWEEN_CALLS)

    except Exception as e:
        print(f"⚠️ Error on {date_str}: {e}")

# Append or save the new data
if all_data:
    df_new = pd.DataFrame(all_data)
    if os.path.exists(SAVE_PATH):
        df_existing = pd.read_csv(SAVE_PATH)
        df_combined = pd.concat([df_existing, df_new], ignore_index=True)
    else:
        df_combined = df_new

    df_combined.to_csv(SAVE_PATH, index=False)
    print(f"✅ Data for {len(all_data)} entries added to {SAVE_PATH}")

else:
    print("⚠️ No new data fetched.")


# Import CSV into code
nasdaq_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv")

# Save the latest date in YYYYMMDD format
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])
latest_date_nasdaq_data = nasdaq_df['date'].max().strftime('%Y%m%d')

[1] Fetching 2025-06-19...
⚠️ No new data fetched.


## Get list of Tickers with Min Market Cap and populate screener_data_df

In [4]:
# Get list of Tickers with a min market cap and a list of common stocks
"""
Returns:
screener_data_df:
  <class 'pandas.core.frame.DataFrame'>
  RangeIndex: 262 entries, 0 to 261
  Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype
  ---  ------                 --------------  -----
 0   code                   262 non-null    object
 1   name                   262 non-null    object
 2   last_day_data_date     262 non-null    object
 3   adjusted_close         262 non-null    float64
 4   refund_1d              262 non-null    float64
 5   refund_1d_p              262 non-null    float64
 6   refund_5d              262 non-null    float64
 7   refund_5d_p            262 non-null    float64
 8   exchange               262 non-null    object
 9   currency_symbol        262 non-null    object
 10  market_capitalization  262 non-null    int64
 11  earnings_share         262 non-null    float64
 12  dividend_yield         145 non-null    float64
 13  sector                 262 non-null    object
 14  industry               262 non-null    object
 15  avgvol_1d              262 non-null    int64
 16  avgvol_200d            262 non-null    float64
 17  Country                262 non-null    object
 18  Exchange               262 non-null    object
 19  Currency               262 non-null    object
 20  Type                   262 non-null    object
 21  last_day_change        262 non-null    float64

"""

# -------------------- CONFIG --------------------
API_KEY = '68433aff09ea73.10710364'  # Replace with your EODHD key
MIN_MARKET_CAP = 10_000_000_000  # Changeable: $1B, $10B, etc.
EXCHANGE = 'NASDAQ'
RESULTS_PER_PAGE = 500  # Max per EODHD API
# ------------------------------------------------

def get_filtered_nasdaq_stocks(api_key, min_cap, exchange="NASDAQ"):
    all_data = []
    offset = 0

    while True:
        url = (
            "https://eodhd.com/api/screener"
            f"?api_token={api_key}"
            f"&filters=["
            f'["exchange","=","{exchange}"],'
            #f'["type","=","Common Stock"],'
            f'["market_capitalization",">=",{min_cap}]'
            f"]"
            f"&sort=market_capitalization.desc"
            f"&limit={RESULTS_PER_PAGE}&offset={offset}&fmt=json"
        )

        response = requests.get(url)
        result = response.json()
        batch = result.get("data", [])

        if not batch:
            break

        all_data.extend(batch)
        offset += RESULTS_PER_PAGE

    return pd.DataFrame(all_data)

# Run filter to get screener_data_df
screener_data_df = get_filtered_nasdaq_stocks(API_KEY, MIN_MARKET_CAP)

# Get exchange symbol list with type
meta_url = f'https://eodhd.com/api/exchange-symbol-list/NASDAQ?api_token={API_KEY}&fmt=json'
meta_df = pd.DataFrame(requests.get(meta_url).json())
common_df = meta_df[meta_df['Type'] == 'Common Stock'].copy() # Add .copy() to avoid SettingWithCopyWarning

# Combine screener_data_df with relevant columns from common_df
screener_data_df = pd.merge(
    screener_data_df,
    common_df[['Code', 'Country', 'Exchange', 'Currency', 'Type']],
    left_on='code',
    right_on='Code',
    how='inner'
).drop('Code', axis=1)

# --- Calculate Daily Change ---
# Ensure 'date' column is datetime and sort by date
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])
temp_nasdaq_sorted = nasdaq_df.sort_values(by=['ticker', 'date']).copy()

# Calculate the previous day's adjusted close for each ticker
temp_nasdaq_sorted['prev_adjusted_close'] = temp_nasdaq_sorted.groupby('ticker')['adjusted_close'].shift(1)

# Calculate the daily change as a percentage
temp_nasdaq_sorted['daily_change'] = ((temp_nasdaq_sorted['adjusted_close'] - temp_nasdaq_sorted['prev_adjusted_close']) / temp_nasdaq_sorted['prev_adjusted_close']) * 100

# Get the last day's data for each ticker
last_day_data_per_ticker = temp_nasdaq_sorted.groupby('ticker').tail(1).copy()

# Create a dictionary of the last day's change for each ticker
last_day_changes = last_day_data_per_ticker.set_index('ticker')['daily_change'].to_dict()

# Add the last day change to the screener_data_df
screener_data_df['last_day_change'] = screener_data_df['code'].map(last_day_changes)

## Filter nasdaq data for screener_data_df and type: common

In [5]:
# prompt: I want to filter the Nasdaq stocks to only include tickers that are part of the common_df and the min_market_cap_df. Same this df as a new variable
# Load your EOD data
from datetime import datetime, timedelta
# nasdaq_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/nasdaq_bulk_eod.csv")
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'])

# Now filter nasdaq_df using the combined dataframe and type 'Common Stock'
filtered_nasdaq_df = nasdaq_df[
    nasdaq_df['ticker'].isin(screener_data_df[screener_data_df['Type'] == 'Common Stock']['code'])
]

## Function: Correlation Coeficient for entire stock data set

In [ ]:
#return six month and three month spearman correlations for all unique pairs of stocks

import pandas as pd
from datetime import datetime # Import datetime
from tqdm.notebook import tqdm # Import tqdm

# ## Function: Correlation Coeficient for entire stock data set
def calculate_lagged_correlation(df, lag_days, range_months):
  """
  Calculates the pairwise spearman correlation coefficient between all stocks
  in a DataFrame for a specified period with a given lag.
  The start date is calculated by subtracting range_months from today's date.

  Args:
    df: DataFrame with 'date', 'ticker', and 'adjusted_close' columns.
    lag_days: The number of days to lag the second stock's data.
    range_months: The number of months to include in the analysis period.

  Returns:
    correlation_matrix: A pandas DataFrame with the following
      Index: Tickers
      Columns: Tickers
      Values: Spearman correlation coefficients between stocks
  """
  # Calculate the end date (today's date)
  end_datetime = datetime.now()

  # Calculate the start date for the specified period by subtracting range_months
  start_datetime = end_datetime - pd.DateOffset(months = range_months)


  # Ensure the 'date' column is in datetime format
  df['date'] = pd.to_datetime(df['date'])

  # Filter the DataFrame for the specified date range
  filtered_df = df[(df['date'] >= start_datetime) & (df['date'] <= end_datetime)].copy()

  # Filter out rows where volume is 0
  filtered_df = filtered_df[filtered_df['volume'] > 0].copy()

  # Get unique tickers in the filtered data
  tickers = filtered_df['ticker'].unique()

  #breakpoint()

  # Create an empty DataFrame to store correlation results
  correlation_matrix = pd.DataFrame(index=tickers, columns=tickers, dtype=float)

  # Iterate through all pairs of tickers with a progress bar
  for ticker_a in tqdm(tickers, desc="Calculating correlations"): # Add tqdm here

    # Extract data for each ticker and name the series for clarity
    stock_a_data = filtered_df[filtered_df['ticker'] == ticker_a].set_index('date')['adjusted_close']
    #breakpoint()

    for ticker_b in tickers:
      if ticker_a != ticker_b:
        # Extract data for each ticker and name the series for clarity
        stock_b_data = filtered_df[filtered_df['ticker'] == ticker_b].set_index('date')['adjusted_close']
        #breakpoint()

        # Align the dataframes based on the date index
        # Suffixes will be applied to the 'adjusted_close' column name
        aligned_data = pd.merge(stock_a_data, stock_b_data,
                                left_index=True, right_index=True,
                                how='inner', suffixes=('_A', '_B'))

        # Apply the lag to stock_b_data, referencing the suffixed column name
        lagged_stock_b_data = aligned_data['adjusted_close_B'].shift(lag_days)

        #breakpoint()

        # Calculate correlation, dropping NaN values
        # Using Spearman method and minimum periods
        #correlation = aligned_data['adjusted_close_A'].corr(lagged_stock_b_data, method='spearman', min_periods=100)

        correlation = aligned_data['adjusted_close_A'].corr(lagged_stock_b_data, method='spearman')


        # Store the correlation in the matrix
        correlation_matrix.loc[ticker_a, ticker_b] = correlation

  return correlation_matrix


# Check GoogleDrive for latest_date_nasdaq_data, in not run the correlation function
"""
Returns:
  three_month_spearman_lagged_correlations and six_month_spearman_lagged_correlations: A pandas DataFrame with the following
    Index: Tickers
    Columns: Tickers
    Values: Spearman correlation coefficients between stocks
"""
import pandas as pd
import os
from datetime import datetime, timedelta

# Define the file paths in Google Drive using the determined date
three_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{latest_date_nasdaq_data}_three_month_spearman_lagged_correlation.csv'
six_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/{latest_date_nasdaq_data}_six_month_spearman_lagged_correlation.csv'

# Check if files exist and load them
try:
    three_month_spearman_lagged_correlations = pd.read_csv(three_month_file, index_col=0)
    print(f"Loaded existing file: {three_month_file}")
    six_month_spearman_lagged_correlations = pd.read_csv(six_month_file, index_col=0)
    print(f"Loaded existing file: {six_month_file}")

except FileNotFoundError:
    print("One or both correlation files not found. Calculating correlations...")
    # Calculate correlations if files don't exist
    # Ensure 'filtered_nasdaq_df' is defined from the preceding code
    if 'filtered_nasdaq_df' in locals():
        three_month_spearman_lagged_correlations = calculate_lagged_correlation(filtered_nasdaq_df, lag_days=1, range_months=3)
        six_month_spearman_lagged_correlations = calculate_lagged_correlation(filtered_nasdaq_df, lag_days=1, range_months=6)

        three_month_spearman_lagged_correlations.to_csv(three_month_file)
        print(f"Saved calculated file: {three_month_file}")
        six_month_spearman_lagged_correlations.to_csv(six_month_file)
        print(f"Saved calculated file: {six_month_file}")
    else:
        print("Error: 'filtered_nasdaq_df' is not defined. Please ensure the preceding code ran correctly.")

In [7]:
import pandas as pd

# Define the file paths in Google Drive using the determined date
three_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250617_three_month_spearman_lagged_correlation.csv'
six_month_file = f'/content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250617_six_month_spearman_lagged_correlation.csv'

# Check if files exist and load them

three_month_spearman_lagged_correlations = pd.read_csv(three_month_file, index_col=0)
print(f"Loaded existing file: {three_month_file}")
six_month_spearman_lagged_correlations = pd.read_csv(six_month_file, index_col=0)
print(f"Loaded existing file: {six_month_file}")

Loaded existing file: /content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250617_three_month_spearman_lagged_correlation.csv
Loaded existing file: /content/drive/MyDrive/Colab Notebooks/Correlation_Data/20250617_six_month_spearman_lagged_correlation.csv


## Process correlated Data and get gravitational scores

In [48]:
import pandas as pd
import numpy as np

# Function to process stock correlation data, calculate gravitational forces,
# and filter connections for visualization based on the force.
def process_and_score_stocks(
    six_month_correlations,
    three_month_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent
):
    """
    Processes stock correlation data for a specific source ticker.
    It filters for positive correlations, computes a dynamic impact score (gravitational_force),
    filters connections, and then calculates a final net gravitational force and the
    maximum potential force under ideal conditions.

    Args:
      six_month_correlations: The six-month spearman lagged correlation matrix.
      three_month_correlations: The three-month spearman lagged correlation matrix.
      screener_data_df: DataFrame with additional stock information.
      source_ticker: The ticker symbol for which to process data.
      min_nodes: Minimum number of correlated stocks to return.
      max_nodes: Maximum number of correlated stocks to return.
      threshold_percent: A percentage (0.0 to 1.0) of the maximum force to use as a filter.

    Returns:
      processed_data_df: A pandas DataFrame with processed data for visualization.
      source_data_df: A pandas DataFrame containing the net_gravitational_force,
                      max_potential_force, and gravitational_impact for the source ticker,
                      along with the source ticker's market cap influence and source_planet_radius.
    """
    # --- Data Unpivoting and Initial Setup ---
    # Start with the 6-month correlation data as the base
    correlation_df = six_month_correlations.rename_axis('source', axis=0)
    grouped_correlation_data = correlation_df.stack().reset_index()
    grouped_correlation_data.columns = ['source', 'target', 'six_month_spearman_correlation']

    grouped_correlation_data = grouped_correlation_data[
        (grouped_correlation_data['source'] != grouped_correlation_data['target']) &
        (grouped_correlation_data['target'] != source_ticker)
    ].copy()

    # --- Filter for the specific source ticker ---
    source_connections = grouped_correlation_data[grouped_correlation_data['source'] == source_ticker].copy()
    if source_connections.empty:
        print(f"No correlation data found for source ticker {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # Add 3-month correlation data before filtering
    source_connections['three_month_spearman_correlation'] = source_connections.apply(
        lambda row: three_month_correlations.loc[row['source'], row['target']] if row['source'] in three_month_correlations.index and row['target'] in three_month_correlations.columns else 0, axis=1
    )

    # We only care about positively correlated stocks for this model in both 6 and 3 month periods
    positive_corr_group = source_connections[
        (source_connections['six_month_spearman_correlation'] > 0) &
        (source_connections['three_month_spearman_correlation'] > 0)
    ].copy()

    if positive_corr_group.empty:
        print(f"No positive correlations found for source ticker {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # --- Enrich Data (before filtering) ---
    # Add market data
    screener_cols_to_add = ['code', 'market_capitalization', 'last_day_change']
    required_screener_cols = ['code', 'market_capitalization', 'last_day_change']
    if not all(col in screener_data_df.columns for col in required_screener_cols):
        missing = [col for col in required_screener_cols if col not in screener_data_df.columns]
        raise ValueError(f"screener_data_df is missing required columns: {missing}")

    screener_info = screener_data_df[screener_cols_to_add].rename(columns={'code': 'target'})
    positive_corr_group = pd.merge(positive_corr_group, screener_info, on='target', how='left')
    positive_corr_group.dropna(subset=['market_capitalization', 'last_day_change'], inplace=True)
    if positive_corr_group.empty:
        print(f"No valid connections after merging screener data for {source_ticker}.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()


    # --- Calculate Dynamic Impact Score (Gravitational Force) ---
    epsilon = 1e-9 # Small value to avoid log(0) issues.
    # Weights for recency bias
    w_3m = 0.6
    w_6m = 0.4
    # "unified_correlation" is a weighted average of recent correlations.
    positive_corr_group['unified_correlation'] = (
        w_3m * positive_corr_group['three_month_spearman_correlation'] +
        w_6m * positive_corr_group['six_month_spearman_correlation']
    )

    # Calculate a market cap influence score scaled between 0 and 1 for target stocks.
    positive_corr_group['Market Cap'] = positive_corr_group['market_capitalization']

    # --- Calculate source ticker's market cap and log cap ---
    source_screener_info = screener_data_df[screener_data_df['code'] == source_ticker]
    source_market_cap = source_screener_info['market_capitalization'].iloc[0] if not source_screener_info.empty and 'market_capitalization' in source_screener_info.columns else epsilon
    source_log_cap = np.log(max(source_market_cap, epsilon))


    # Calculate log market caps for all relevant tickers (source and targets)
    all_market_caps = positive_corr_group['Market Cap'].tolist()
    all_market_caps.append(source_market_cap) # Include source market cap

    log_caps = np.log(pd.Series(all_market_caps).clip(lower=epsilon))

    min_log_cap, max_log_cap = log_caps.min(), log_caps.max()
    log_cap_range = max_log_cap - min_log_cap

    # Calculate market cap influence for target stocks
    if log_cap_range > 0:
        positive_corr_group['market_cap_influence'] = np.log(positive_corr_group['Market Cap'].clip(lower=epsilon))
    else:
        positive_corr_group['market_cap_influence'] = 20 # Neutral value if all caps are the same


    # The `gravitational_force` is a product of recent correlation strength and market influence.
    # Modified: Increased the influence of unified_correlation by multiplying by a factor
    correlation_weight_factor = 1.0 # Factor to increase the influence of unified_correlation
    positive_corr_group['gravitational_force'] = (
        (positive_corr_group['unified_correlation'] * correlation_weight_factor) * # Multiply unified_correlation by a factor
        positive_corr_group['market_cap_influence']
    )

    # --- Apply Filtering ---
    max_abs_force = positive_corr_group['gravitational_force'].abs().max()
    if pd.isna(max_abs_force) or max_abs_force == 0:
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    force_threshold = max_abs_force * threshold_percent
    filtered_by_force_threshold = positive_corr_group[positive_corr_group['gravitational_force'].abs() >= force_threshold].copy()

    # Enforce min/max node constraints
    if len(filtered_by_force_threshold) < min_nodes:
        final_filtered_df = positive_corr_group.sort_values(by='gravitational_force', key=abs, ascending=False).head(min_nodes).copy()
    elif len(filtered_by_force_threshold) > max_nodes:
        final_filtered_df = filtered_by_force_threshold.sort_values(by='gravitational_force', key=abs, ascending=False).head(max_nodes).copy()
    else:
        final_filtered_df = filtered_by_force_threshold.copy()

    if final_filtered_df.empty:
        print(f"No connections remained for {source_ticker} after filtering.")
        # Return empty dataframes when no data is found
        return pd.DataFrame(), pd.DataFrame()

    # --- Calculate Final Net Force and Visualization Parameters ---
    final_filtered_df['Daily Change'] = final_filtered_df['last_day_change']

    final_filtered_df['signed_gravitational_force'] = final_filtered_df.apply(
        lambda row: row['gravitational_force'] if row['Daily Change'] >= 0 else -row['gravitational_force'],
        axis=1
    )

    net_gravitational_force = final_filtered_df['signed_gravitational_force'].sum()
    max_potential_force = final_filtered_df['market_cap_influence'].sum()

    # --- Calculate Visualization Parameters ---
    min_corr, max_corr = final_filtered_df['gravitational_force'].min(), final_filtered_df['gravitational_force'].max()
    corr_range = max_corr - min_corr if max_corr > min_corr else 1.0
    # MODIFIED: Reverse the scaling for Orbital Radius
    if corr_range > 0:
        final_filtered_df['Orbital Radius'] = 1 - ((final_filtered_df['gravitational_force'] - min_corr) / corr_range)
    else:
        final_filtered_df['Orbital Radius'] = 0.5 # Neutral value if all forces are the same

    # -----Calculate Planet Radius------
    # Combine all market caps to find the true min and max for normalization
    all_caps = pd.concat([
        final_filtered_df['Market Cap'],
        pd.Series([source_market_cap]) # Make sure source_market_cap is a Series
    ], ignore_index=True)

    # Calculate the log, clipping to avoid errors with zero
    log_all_caps = np.log(all_caps.clip(lower=epsilon))

    # Find the min and max from the complete set of data
    min_log_cap = log_all_caps.min()
    max_log_cap = log_all_caps.max()
    log_cap_range = max_log_cap - min_log_cap

    # Now, apply the normalization ONLY to the DataFrame's data
    # using the min/max from the combined set
    if log_cap_range > 0:
        # We are calculating log on just the dataframe column now
        log_df_caps = np.log(final_filtered_df['Market Cap'].clip(lower=epsilon))
        final_filtered_df['Planet Radius'] = (log_df_caps - min_log_cap) / log_cap_range
    else:
        # If all values are the same, assign a default radius
        final_filtered_df['Planet Radius'] = 0.5

    # Calculate source_planet_radius using the same min/max log caps from the targets and source.
    if log_cap_range > 0:
        source_planet_radius = (source_log_cap - min_log_cap) / log_cap_range
    else:
        source_planet_radius = 0.5 # Neutral value if all caps are the same

    # --- Final Cleanup and Column Selection ---
    # "gravitational_percent" shows the relative % contribution of each stock.
    final_filtered_df['gravitational_percent'] = (final_filtered_df['signed_gravitational_force'] / final_filtered_df['gravitational_force'].sum()) * 100

    final_columns = [
        'source', 'target', 'Daily Change', 'six_month_spearman_correlation',
        'three_month_spearman_correlation', 'unified_correlation',
        'Orbital Radius', 'Market Cap', 'Planet Radius', 'market_cap_influence',
        'gravitational_force', 'signed_gravitational_force', 'gravitational_percent'
    ]


    gravitational_impact = (net_gravitational_force / max_potential_force) * 100 if max_potential_force > 0 else 0

    # Use the same min_log_cap and log_cap_range from target stocks for scaling
    source_market_cap_influence = 20 if log_cap_range <= 0 else (source_log_cap)

    # Create source_data_df
    source_data_df = pd.DataFrame([{
        'ticker': source_ticker,
        'net_gravitational_force': net_gravitational_force,
        'max_potential_force': max_potential_force,
        'gravitational_impact': gravitational_impact,
        'source_market_cap_influence': source_market_cap_influence, # Add the source influence
        'source_planet_radius': source_planet_radius # Add the source planet radius
    }])


    for col in final_columns:
        if col not in final_filtered_df.columns:
            final_filtered_df[col] = np.nan

    processed_data_df = final_filtered_df[final_columns].copy()

    return processed_data_df, source_data_df


## ---------- MODIFIED: Run App ---------------
min_nodes = 5
max_nodes = 30
threshold_percent = 0.9

# User input Ticker
source_ticker = 'AAPL'

# Process the data for the network diagram
processed_data_df, source_data_df = process_and_score_stocks(
    six_month_spearman_lagged_correlations,
    three_month_spearman_lagged_correlations,
    screener_data_df,
    source_ticker,
    min_nodes,
    max_nodes,
    threshold_percent,
    )

# Extract the scalar values from the source_data_df for plotting
net_gravitational_force = source_data_df['net_gravitational_force'].iloc[0]
max_potential_force = source_data_df['max_potential_force'].iloc[0]
gravitational_impact = source_data_df['gravitational_impact'].iloc[0]
market_cap_influence = source_data_df['source_market_cap_influence'].iloc[0]
source_planet_radius = source_data_df['source_planet_radius'].iloc[0]

print(f"Net Gravitational Force: {net_gravitational_force:.2f}")
print(f"Max Potential Gravitational Force: {max_potential_force:.2f}")
print(f"Net Gravitaional Impact: {gravitational_impact:.2f}%")
print(f"Source Market Cap Influence: {market_cap_influence}")
print(f"Source Planet Radius: {source_planet_radius}")
print('----------------------------------')
processed_data_df

Net Gravitational Force: -53.22
Max Potential Gravitational Force: 119.93
Net Gravitaional Impact: -44.38%
Source Market Cap Influence: 28.717417120686775
Source Planet Radius: 1.0
----------------------------------


,source,target,Daily Change,six_month_spearman_correlation,three_month_spearman_correlation,unified_correlation,Orbital Radius,Market Cap,Planet Radius,market_cap_influence,gravitational_force,signed_gravitational_force,gravitational_percent
133,AAPL,TEAM,-0.345493,0.856194,0.770069,0.804519,0.000000,52585234432,0.269891,24.685701,19.860116,-19.860116,-22.911403
111,AAPL,PCAR,-2.051393,0.785738,0.624268,0.688856,0.869736,48663707648,0.255856,24.608199,16.951511,-16.951511,-19.555924
37,AAPL,ENTG,1.194585,0.821354,0.654469,0.721223,0.936270,11847050240,0.000000,23.195345,16.729006,16.729006,19.299233
145,AAPL,VNOM,-1.400290,0.803521,0.657139,0.715692,0.967155,12268126208,0.006325,23.230270,16.625720,-16.625720,-19.180078
54,AAPL,GEHC,-0.498270,0.754941,0.633845,0.682283,1.000000,32573937664,0.183162,24.206778,16.515879,-16.515879,-19.053362


##List of Top Predictions

In [49]:
import pandas as pd
top_gravitational_impacts = []

# Ensure unified_correlation_df is created from six_month_spearman_lagged_correlations
# assuming unified_correlation_df is meant to represent the base correlations
# based on the usage in the provided code for process_and_score_stocks.
# If unified_correlation_df should be different, please adjust this.
unified_correlation_df = six_month_spearman_lagged_correlations.copy()


# Iterate through each ticker in the unified_correlation_df
for ticker in tqdm(unified_correlation_df.index, desc="Processing tickers for gravitational impact"):
    try:
        # Run each ticker through the process_and_score_stocks function
        # Note: The function previously returned processed_data_df and prediction_score.
        # It now returns processed_data_df, net_gravitational_force, max_potential_force, gravitational_impact
        # We need to unpack the new return values.
        processed_df, source_data = process_and_score_stocks(
            six_month_spearman_lagged_correlations, # Pass the 6-month correlation
            three_month_spearman_lagged_correlations, # Pass the 3-month correlation
            screener_data_df,
            ticker, # Use the current ticker as source_ticker
            min_nodes,
            max_nodes,
            threshold_percent,
        )

        # Append the results to the list
        # Access the scalar values from the returned source_data DataFrame
        if not source_data.empty:
            top_gravitational_impacts.append({
                'ticker': ticker,
                'net_gravitational_force': source_data['net_gravitational_force'].iloc[0],
                'max_potential_force': source_data['max_potential_force'].iloc[0],
                'gravitational_impact': source_data['gravitational_impact'].iloc[0]
            })
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")

# Create a DataFrame from the results
gravitational_impact_df = pd.DataFrame(top_gravitational_impacts)

# Sort by gravitational_impact in descending order for top positive impacts
top_positive_impacts = gravitational_impact_df.sort_values(by='gravitational_impact', ascending=False).head(10).reset_index(drop=True)

# Sort by gravitational_impact in ascending order for top negative impacts
top_negative_impacts = gravitational_impact_df.sort_values(by='gravitational_impact', ascending=True).head(10).reset_index(drop=True)


# Display the tables
print("Top 10 Positive Gravitational Impact Tickers:")
print(top_positive_impacts[['ticker', 'net_gravitational_force', 'max_potential_force', 'gravitational_impact']].to_string(index=False, formatters={'net_gravitational_force': '{:.2f}'.format, 'max_potential_force': '{:.2f}'.format, 'gravitational_impact': '{:.2f}'.format}))

print("\nTop 10 Negative Gravitational Impact Tickers:")
print(top_negative_impacts[['ticker', 'net_gravitational_force', 'max_potential_force', 'gravitational_impact']].to_string(index=False, formatters={'net_gravitational_force': '{:.2f}'.format, 'max_potential_force': '{:.2f}'.format, 'gravitational_impact': '{:.2f}'.format}))

Processing tickers for gravitational impact:   0%|          | 0/259 [00:00<?, ?it/s]

No correlation data found for source ticker CHYM.
Top 10 Positive Gravitational Impact Tickers:
ticker net_gravitational_force max_potential_force gravitational_impact
  NICE                  115.80              138.94                83.35
  ASML                  115.39              139.25                82.86
  CTSH                  108.91              131.95                82.54
  NDAQ                   97.16              163.37                59.48
  CRDO                   98.58              168.21                58.61
   APP                   97.41              167.55                58.14
  HOOD                   92.55              161.86                57.18
   CEG                   93.56              165.07                56.68
   WDC                   76.93              139.32                55.22
  FLEX                   76.59              139.08                55.07

Top 10 Negative Gravitational Impact Tickers:
ticker net_gravitational_force max_potential_force gravitational_

#Dash App with Solar System

##Setup

In [10]:
# Dash App Setup

#Terminal Command
#ngrok config add-authtoken 2yPWVwCXJDsY9JkhLhnrn549R5E_5b2uFuh7R4ER3fLJWLU59

# Install necessary libraries (run this cell first)
!pip install -q dash dash-bootstrap-components plotly pyngrok

# --- Import necessary libraries ---
from dash import Dash, html, dcc, Input, Output # We need dcc for interactive components and Input/Output for callbacks
import os
from pyngrok import ngrok
from google.colab import userdata

# # --- Configure Ngrok for public access ---
# NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
# os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN

# --- Configure Ngrok for public access ---
# This part remains the same to create a public URL for your app
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
    # Use pyngrok to set the auth token programmatically
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
except userdata.SecretNotFoundError:
    print("NGROK_AUTH_TOKEN not found in Colab secrets. Please add it.")
    # You might want to handle this case, e.g., by exiting or using a default token if applicable
    exit()


# Define the port your Dash app will run on.
DASH_PORT = 8051

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.5 MB/s eta 0:00:00


In [ ]:
# Kills all ngrok processes that pyngrok started
print("Attempting to kill all active ngrok tunnels...")
ngrok.kill()
print("All ngrok tunnels should now be terminated.")

# Add a short delay to ensure the process is killed
import time
time.sleep(3)

Attempting to kill all active ngrok tunnels...
All ngrok tunnels should now be terminated.


In [11]:
# Create a public ngrok tunnel to the specified port.
print("Establishing Ngrok tunnel...")
public_url = ngrok.connect(DASH_PORT)
print(f"🚀 Your Dash app will be accessible at: {public_url}")

Establishing Ngrok tunnel...
🚀 Your Dash app will be accessible at: NgrokTunnel: "https://09d4-34-168-231-68.ngrok-free.app" -> "http://localhost:8051"


## Dash App

In [50]:
# The Financial Planetarium
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations, especially for generating coordinates
import plotly.graph_objects as go  # The main library for creating interactive plots
import matplotlib.colors as mcolors # For creating custom color maps and normalizing color scales
from dash import Dash, dcc, html, callback_context   # The framework for building the web application interface
from dash.dependencies import Input, Output, State, ALL # To create interactivity in the Dash app (callbacks)
from dash.exceptions import PreventUpdate
import json


# --- Helper Function to Create Wireframe Spheres ---
def create_wireframe_sphere(center_x, center_y, center_z, radius, color, n_lat=20, n_lon=20):
    """
    Generates the coordinate data for a 3D wireframe sphere for a Plotly graph.
    This function calculates points along latitude and longitude lines to give the
    appearance of a sphere without rendering a solid surface.

    Args:
        center_x, center_y, center_z (float): The center coordinates of the sphere.
        radius (float): The radius of the sphere.
        color (str): The color of the wireframe lines (in a format Plotly understands, e.g., hex code).
        n_lat (int): The number of latitude lines to draw. More lines create a denser sphere.
        n_lon (int): The number of longitude lines to draw. More lines create a denser sphere.

    Returns:
        tuple: A tuple containing three lists (x_lines, y_lines, z_lines) of coordinates
               for the sphere's lines. `None` values are used to separate the lines.
    """
    # Initialize lists to hold the coordinates for all line segments
    x_lines, y_lines, z_lines = [], [], []

    # --- Generate latitude lines (horizontal rings) ---
    # Loop through each latitude angle from the North Pole to the South Pole
    for i in range(n_lat + 1):
        # Calculate the latitude angle. np.pi/2 is the North Pole, -np.pi/2 is the South Pole.
        lat_angle = np.pi / 2 - (i * np.pi / n_lat)
        z = radius * np.sin(lat_angle) # Z coordinate is constant for a given latitude
        r = radius * np.cos(lat_angle) # The radius of the horizontal ring at this latitude

        # Loop through each longitude to draw the points on the ring
        for j in range(n_lon + 1):
            lon_angle = j * 2 * np.pi / n_lon # Go a full circle (2*pi)
            x = r * np.cos(lon_angle) # Calculate x based on the ring radius and longitude
            y = r * np.sin(lon_angle) # Calculate y based on the ring radius and longitude

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full latitude line. This tells Plotly to stop drawing
        # the current line and start a new one, preventing lines from connecting across the sphere.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    # --- Generate longitude lines (vertical meridians) ---
    # Loop through each longitude angle
    for i in range(n_lon + 1):
        lon_angle = i * 2 * np.pi / n_lon # The angle for this specific meridian

        # Loop through each latitude to draw the points along the vertical line
        for j in range(n_lat + 1):
            lat_angle = np.pi / 2 - (j * np.pi / n_lat) # Go from North to South Pole
            z = radius * np.sin(lat_angle) # Calculate z for this point on the meridian
            r = radius * np.cos(lat_angle) # Calculate the horizontal distance from the z-axis

            # Calculate x and y based on the constant longitude angle
            x = r * np.cos(lon_angle)
            y = r * np.sin(lon_angle)

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full longitude line to start a new line.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    return x_lines, y_lines, z_lines

# --- Define Color Mapping (Global for use in layout and plotting) ---
colors = ['#FF0000', '#F04435', '#E47C61', '#D9B28B', '#A5BA6F', '#73C255', '#33CC33']
cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors)

# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df):
    """
    Creates the main 3D solar system visualization for a given central stock (source_ticker).

    Args:
        source_ticker (str): The ticker symbol for the central star of the system.
        processed_data_df (pd.DataFrame): DataFrame with data about connections (planets).
        source_data_df (pd.DataFrame): DataFrame with data about the source stocks (stars).
        screener_data_df (pd.DataFrame): DataFrame with general financial info for all tickers.

    Returns:
        go.Figure: A Plotly Figure object containing the complete 3D visualization.
    """
    # Filter the data to get only the connections for the selected source_ticker
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()

    # Get the specific row of data for the source_ticker itself
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    # If there's no data for the selected ticker, return an empty figure with a message
    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='black', font_color='white')

    # Extract the source info into a pandas Series for easier access
    source_info = source_info_row.iloc[0]

    # Initialize the Plotly Figure
    fig = go.Figure()

    # Create a dictionary to store the 3D positions (x, y, z) of each stock (node).
    # The source ticker (the "star") is always placed at the origin (0, 0, 0).
    pos = {source_ticker: (0, 0, 0)}

    # Filter out any connections where the target is the source itself
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)

    # List to store the calculated visual radii for the orbital rings
    radii_for_rings = []

    # Calculate positions for the "planets" (connected tickers) if any exist
    if num_connections > 0:
        # --- Scale orbital distances for better visualization ---
        # Get the original orbital radii from the data
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()

        # Avoid division by zero if all radii are the same
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0

        # Define the range for the visual radii in the plot (e.g., from 2 to 8 units)
        min_visual_radius, max_visual_radius = 2, 8.0
        visual_range = max_visual_radius - min_visual_radius

        # Generate evenly spaced angles to position the planets around the star
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        # Loop through each connected "planet" to calculate its position
        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            # Scale the original radius to the new visual range
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)

            # Get the angle for this planet
            theta = thetas[i]

            # Calculate and store the (x, y, z) position using polar coordinates. Z is 0 for a flat system.
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    # Draw a dotted circle for each unique orbital radius
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100) # 100 points for a smooth circle
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(100, 100, 100, 0.5)', width=1, dash='dot'), # Faint, dotted line
            hoverinfo='none' # These rings should not be interactive
        ))

    # Create normalizers to map financial data values to the 0-1 range for the colormap
    norm_daily_change = mcolors.Normalize(vmin=-5, vmax=5) # For planet colors
    norm_gravitational_impact = mcolors.Normalize(vmin=-100, vmax=100) # For star color

    # List to hold annotations (text labels) for the scene
    scene_annotations = []

    # --- Loop Through Each Node (Star and Planets) to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        # Get financial info for the current node from the screener dataframe
        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue # Skip if no data is found
        screener_info = screener_info_row.iloc[0]

        # Format the market cap for readability (Billions or Trillions)
        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"


        # Define the range for the visual size of the spheres
        min_visual_size, max_visual_size = 0.3, 1.0

        # --- Configure Node Properties (Text, Color, Size) ---
        if is_source: # If the node is the central star
            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on gravitational impact
            node_color = mcolors.to_hex(cmap(norm_gravitational_impact(source_info['gravitational_impact'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))

        else: # If the node is a planet
            processed_info_row = ticker_connections[ticker_connections['target'] == node_name]
            if processed_info_row.empty: continue # Skip if no processed data
            processed_info = processed_info_row.iloc[0]

            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on daily price change
            node_color = mcolors.to_hex(cmap(norm_daily_change(processed_info['Daily Change'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))

        # --- Create and add the wireframe sphere for the current node ---
        sphere_x, sphere_y, sphere_z = create_wireframe_sphere(center_x, center_y, center_z, radius, node_color)
        fig.add_trace(go.Scatter3d(
            x=sphere_x, y=sphere_y, z=sphere_z,
            mode='lines',
            line=dict(color=node_color, width=2),
            hoverinfo='none' # The wireframe itself doesn't have a hover label
        ))

        # --- Dynamic Label Positioning using Scene Annotations ---
        # Add a text label for each node that is rendered on top of the 3D scene.
        scene_annotations.append(
            dict(
                x=center_x, y=center_y, z=center_z, # Position the label
                text=f"<b>{node_name}</b>",
                showarrow=False,
                # For planets, use a black font on a colored background. For the star, use white on a transparent background.
                font=dict(color='white' if not is_source else '#FFFFFF', size=16),
                # For planets, set the background color to the planet's color. For the star, it's transparent.
                bgcolor="rgba(0,0,0,0)"
            )
        )

        # --- HOVER INFO TRICK ---
        # Scene annotations don't support rich hover text. So, we add an INVISIBLE marker at the
        # center of each sphere. We make this marker large enough to be easily hovered over and
        # assign the detailed text to its `hovertext` property.
        fig.add_trace(go.Scatter3d(
            x=[center_x], y=[center_y], z=[center_z],
            mode='markers',
            marker=dict(size=radius*20, color='rgba(0,0,0,0)'), # Invisible, but sized for hovering
            hoverinfo='text',
            hovertext=text # This is the detailed text that will appear on hover
        ))

    # --- Configure Final Layout ---
    # Get the full company name for the title
    company_name_row = screener_data_df[screener_data_df['code'] == source_ticker]
    company_name = company_name_row['name'].iloc[0] if not company_name_row.empty else source_ticker # Use ticker if name not found
    fig.update_layout(
        title=f'Financial Solar System for {company_name} ({source_ticker})',
        title_x=0.5, # Center the title
        scene=dict(
            # Hide the 3D axes, grid, and background for a cleaner "space" look
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            # Set the camera's position (eye) and the point it looks at (center)
            camera=dict(
                eye=dict(x=0.97, y=0.97, z=0.5),      # Camera position is below the plane
                center=dict(x=0, y=0, z=-0.3)       # Look at a point slightly above the origin
            ),
            # Use 'data' aspect mode to ensure spheres look like spheres and not ellipsoids
            aspectmode='data',
            # Add the text annotations to the scene
            annotations=scene_annotations
        ),
        margin=dict(l=0, r=0, b=0, t=40), # Reduce margins to maximize plot area
        showlegend=False, # We don't need a legend for this plot
        paper_bgcolor='black', # Background of the entire figure area
        plot_bgcolor='black', # Background of the plotting area itself
        # Apply the font to the entire plot (title, etc.) and increase size
        font=dict(
            family="'VT323', monospace",
            color='#39FF14',
            size=18
        )
    )
    return fig

# --- Dash App Definition ---
# Initialize the Dash application
app = Dash(__name__)

# --- Styling Dictionaries for the UI ---
# Reusable styles for a consistent "terminal" look and feel using the VT323 font
terminal_font = {'fontFamily': "'VT323', monospace", 'color': '#39FF14', 'fontSize': '18px'}
terminal_container_style = {'backgroundColor': 'black', 'border': '1px solid #39FF14', 'padding': '10px 15px', 'margin': '10px', 'borderRadius': '5px'}
terminal_header_style = {'color': '#00FFFF', 'textAlign': 'center', 'text-decoration': 'underline', 'marginTop': 0, 'marginBottom': '10px','fontFamily': "'VT323', monospace", 'fontSize': '22px'}
button_style_green = {**terminal_font, 'backgroundColor': '#33CC33', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}
button_style_red = {**terminal_font, 'backgroundColor': '#FF0000', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}


# Assume screener_data_df is loaded and available in the global scope
# Create the list of options for the dropdown menu from the screener data
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
# Set a default ticker for the app to load initially
default_ticker = 'AAPL' if 'AAPL' in screener_data_df['code'].unique() else screener_data_df['code'].iloc[0]

# --- App Layout ---
# This defines the HTML structure of the web page.
app.layout = html.Div(style={'backgroundColor': 'black', 'color': '#39FF14', **terminal_font}, children=[
    # Import the Google Font
    html.Link(
        rel='stylesheet',
        href='https://fonts.googleapis.com/css2?family=VT323&display=swap'
    ),

    # Main title of the application
    html.H1("The Financial Planetarium", style={'textAlign': 'center', 'paddingTop': '20px', **terminal_font, 'fontSize': '36px', 'color': '#00FFFF'}),

    # Container for the dropdown menu
    html.Div([
        html.Label("Select a Stock:", style={'marginRight': '10px', **terminal_font}),
        dcc.Dropdown(
            id='ticker-dropdown', # Unique ID for this component
            options=ticker_options, # The list of tickers to choose from
            value=default_ticker, # The initial value
            style={'width': '300px', 'display': 'inline-block', **terminal_font, 'color': 'black'}
        ),
    ], style={'textAlign': 'center', 'padding': '20px'}),

    # Main content area with the graph and info tables, laid out horizontally
    html.Div([
        # --- Left Panel: Top Predictions ---
        html.Div([
            html.H3("Top Predictions", style=terminal_header_style),
            # Container for the two columns of buttons
            html.Div([
                # Column 1: Positive Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.2f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_green
                    ) for index, row in top_positive_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginRight': '5px'}
                ),
                # Column 2: Negative Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.1f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_red
                    ) for index, row in top_negative_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginLeft': '5px'}
                ),
            ], style={'display': 'flex', 'flexDirection': 'row'})
        ], style={**terminal_container_style, 'width': '20%', 'alignSelf': 'flex-start'}),

        # --- Center Panel: The Graph ---
        dcc.Graph(id='network-graph', style={'height': '50vh', 'width': '55%'}),

        # --- Right Panel: Info and Legend ---
        html.Div([
            html.Div(id='star-info-table', style=terminal_container_style),
            html.Div([
                html.H3("Legend", style=terminal_header_style),
                html.Div(style={
                    'height': '20px',
                    'width': '100%',
                    'background': f"linear-gradient(to right, {', '.join(colors)})",
                    'borderRadius': '3px'
                }),
                html.Div([
                    html.Span("Decrease", style={'textAlign': 'left', 'flex': 1}),
                    html.Span("Stable", style={'textAlign': 'center', 'flex': 1}),
                    html.Span("Increase", style={'textAlign': 'right', 'flex': 1})
                ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '5px'})
            ], style=terminal_container_style)
        ], style={'width': '25%', 'display': 'flex', 'flexDirection': 'column'})

    ], style={'display': 'flex', 'flexDirection': 'row'}),

    # A Div below the main content to hold the "Planet Information" table
    html.Div(id='planet-info-table', style={**terminal_container_style, 'marginTop': '20px'}),
])

# --- Callback for Prediction Buttons ---
@app.callback(
    Output('ticker-dropdown', 'value'),
    Input({'type': 'prediction-button', 'ticker': ALL}, 'n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_from_button(n_clicks):
    # Check if any button was actually clicked
    if not any(n_clicks):
        raise PreventUpdate

    # Get the ID of the button that was clicked
    ctx = callback_context
    button_id_str = ctx.triggered[0]['prop_id'].split('.')[0]
    button_id = json.loads(button_id_str)

    # Return the ticker from the button's ID to update the dropdown
    return button_id['ticker']

# --- Callback to Update All Components ---
@app.callback(
    [Output('network-graph', 'figure'),
     Output('star-info-table', 'children'),
     Output('planet-info-table', 'children')],
    [Input('ticker-dropdown', 'value')]
)
def update_all_components(selected_ticker):
    """
    This function is triggered whenever the user selects a new ticker from the dropdown.
    It re-calculates and re-generates all the visual components of the app.
    """
    # NOTE: The data processing part is assumed to happen here.
    # In a real application, you would load or calculate your DataFrames.
    # The following line is an example and is commented out because it depends
    # on variables not defined in this standalone script.

    global processed_data_df, source_data_df, screener_data_df, six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations,
        three_month_spearman_lagged_correlations,
        screener_data_df,
        selected_ticker,
        min_nodes,
        max_nodes,
        threshold_percent
    )


    # Check if selected_ticker exists in screener_data_df and source_data_df
    if selected_ticker not in screener_data_df['code'].unique() or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='black', font_color='#39FF14', font_family="'VT323', monospace")
        empty_star_table = [html.H3("Star Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        empty_planet_table = [html.H3("Planet Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        return empty_fig, empty_star_table, empty_planet_table


    # Generate the 3D plot using the main visualization function
    fig = solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df)

    # --- Generate Star Information Table ---
    # Fetch the data for the selected star
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    market_cap = star_info_screener.get('market_capitalization', 0)
    market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"

    # Build the HTML components for the star info box
    star_table_children = [
        html.H3("Star Information", style=terminal_header_style),
        html.P(f"Name: {star_info_screener['name']} ({selected_ticker})", style=terminal_font),
        html.P(f"Market Cap: {market_cap_str}", style=terminal_font),
        html.P(f"Gravitational Impact: {star_info_source['gravitational_impact']:.2f}%", style=terminal_font),
    ]

    # --- Generate Planet Information Table ---
    # Get the data for all planets orbiting the selected star
    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker]

    # --- Style definitions for the planet table ---
    # Style for the table header (solid green bar, black bold text, left-aligned)
    table_header_style = {
        **terminal_font, # Inherit base font styles
        'backgroundColor': '#39FF14',
        'color': 'black',
        'fontWeight': 'bold',
        'textAlign': 'left',
        'padding': '8px'
    }
    # Style for the table data cells (left-aligned text with padding)
    table_cell_style = {
        **terminal_font, # Inherit base font styles
        'textAlign': 'left',
        'padding': '8px',
        'borderBottom': '1px solid rgba(57, 255, 20, 0.3)' # Faint green line for row separation
    }

    # Define the table header using the new style
    table_header = [html.Thead(html.Tr([html.Th(col, style=table_header_style) for col in ["Ticker", "Name", "Market Cap", "Daily Change", "Correlation", "Gravitational Force"]]))]

    table_rows = []
    # Loop through the planet data to create a row for each one, applying the cell style
    for i, planet_row in planets_df.iterrows():
        screener_info_row = screener_data_df[screener_data_df['code'] == planet_row['target']]
        if screener_info_row.empty: continue # Skip if no screener data for planet

        screener_info = screener_info_row.iloc[0]
        planet_mcap_str = f"${screener_info['market_capitalization']/1e12:.2f}T" if screener_info['market_capitalization'] > 1e12 else f"${screener_info['market_capitalization']/1e9:.2f}B"
        table_rows.append(html.Tr([
            html.Td(planet_row['target'], style=table_cell_style),
            html.Td(screener_info['name'], style=table_cell_style),
            html.Td(planet_mcap_str, style=table_cell_style),
            html.Td(f"{planet_row['Daily Change']:.2f}%", style=table_cell_style),
            html.Td(f"{planet_row['unified_correlation']:.3f}", style=table_cell_style),
            html.Td(f"{planet_row['signed_gravitational_force']:.2f}", style=table_cell_style),
        ]))

    # Combine the header and rows into a full table
    planet_table_children = [
        html.H3("Planet Information", style=terminal_header_style),
        html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse', 'borderSpacing': 0})
    ]

    # Return the generated figure and table contents to the appropriate 'Output' components
    return fig, star_table_children, planet_table_children

# --- Run the App ---
# This line is standard for running a Python script directly.
# The `if __name__ == '__main__':` block ensures this code only runs
# when the script is executed, not when it's imported as a module.
if __name__ == '__main__':
    # Starts the web server. `debug=True` allows for hot-reloading and provides error messages in the browser.
    app.run(debug=True, port=8051)


<IPython.core.display.Javascript object>

#Testing Grounds

In [39]:
# The Financial Planetarium
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations, especially for generating coordinates
import plotly.graph_objects as go  # The main library for creating interactive plots
import matplotlib.colors as mcolors # For creating custom color maps and normalizing color scales
from dash import Dash, dcc, html, callback_context   # The framework for building the web application interface
from dash.dependencies import Input, Output, State, ALL # To create interactivity in the Dash app (callbacks)
from dash.exceptions import PreventUpdate
import json


# --- Helper Function to Create Wireframe Spheres ---
def create_wireframe_sphere(center_x, center_y, center_z, radius, color, n_lat=20, n_lon=20):
    """
    Generates the coordinate data for a 3D wireframe sphere for a Plotly graph.
    This function calculates points along latitude and longitude lines to give the
    appearance of a sphere without rendering a solid surface.

    Args:
        center_x, center_y, center_z (float): The center coordinates of the sphere.
        radius (float): The radius of the sphere.
        color (str): The color of the wireframe lines (in a format Plotly understands, e.g., hex code).
        n_lat (int): The number of latitude lines to draw. More lines create a denser sphere.
        n_lon (int): The number of longitude lines to draw. More lines create a denser sphere.

    Returns:
        tuple: A tuple containing three lists (x_lines, y_lines, z_lines) of coordinates
               for the sphere's lines. `None` values are used to separate the lines.
    """
    # Initialize lists to hold the coordinates for all line segments
    x_lines, y_lines, z_lines = [], [], []

    # --- Generate latitude lines (horizontal rings) ---
    # Loop through each latitude angle from the North Pole to the South Pole
    for i in range(n_lat + 1):
        # Calculate the latitude angle. np.pi/2 is the North Pole, -np.pi/2 is the South Pole.
        lat_angle = np.pi / 2 - (i * np.pi / n_lat)
        z = radius * np.sin(lat_angle) # Z coordinate is constant for a given latitude
        r = radius * np.cos(lat_angle) # The radius of the horizontal ring at this latitude

        # Loop through each longitude to draw the points on the ring
        for j in range(n_lon + 1):
            lon_angle = j * 2 * np.pi / n_lon # Go a full circle (2*pi)
            x = r * np.cos(lon_angle) # Calculate x based on the ring radius and longitude
            y = r * np.sin(lon_angle) # Calculate y based on the ring radius and longitude

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full latitude line. This tells Plotly to stop drawing
        # the current line and start a new one, preventing lines from connecting across the sphere.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    # --- Generate longitude lines (vertical meridians) ---
    # Loop through each longitude angle
    for i in range(n_lon + 1):
        lon_angle = i * 2 * np.pi / n_lon # The angle for this specific meridian

        # Loop through each latitude to draw the points along the vertical line
        for j in range(n_lat + 1):
            lat_angle = np.pi / 2 - (j * np.pi / n_lat) # Go from North to South Pole
            z = radius * np.sin(lat_angle) # Calculate z for this point on the meridian
            r = radius * np.cos(lat_angle) # Calculate the horizontal distance from the z-axis

            # Calculate x and y based on the constant longitude angle
            x = r * np.cos(lon_angle)
            y = r * np.sin(lon_angle)

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full longitude line to start a new line.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    return x_lines, y_lines, z_lines

# --- Define Color Mapping (Global for use in layout and plotting) ---
colors = ['#FF0000', '#F04435', '#E47C61', '#D9B28B', '#A5BA6F', '#73C255', '#33CC33']
cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors)

# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df):
    """
    Creates the main 3D solar system visualization for a given central stock (source_ticker).

    Args:
        source_ticker (str): The ticker symbol for the central star of the system.
        processed_data_df (pd.DataFrame): DataFrame with data about connections (planets).
        source_data_df (pd.DataFrame): DataFrame with data about the source stocks (stars).
        screener_data_df (pd.DataFrame): DataFrame with general financial info for all tickers.

    Returns:
        go.Figure: A Plotly Figure object containing the complete 3D visualization.
    """
    # Filter the data to get only the connections for the selected source_ticker
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()

    # Get the specific row of data for the source_ticker itself
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    # If there's no data for the selected ticker, return an empty figure with a message
    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='black', font_color='white')

    # Extract the source info into a pandas Series for easier access
    source_info = source_info_row.iloc[0]

    # Initialize the Plotly Figure
    fig = go.Figure()

    # Create a dictionary to store the 3D positions (x, y, z) of each stock (node).
    # The source ticker (the "star") is always placed at the origin (0, 0, 0).
    pos = {source_ticker: (0, 0, 0)}

    # Filter out any connections where the target is the source itself
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)

    # List to store the calculated visual radii for the orbital rings
    radii_for_rings = []

    # Calculate positions for the "planets" (connected tickers) if any exist
    if num_connections > 0:
        # --- Scale orbital distances for better visualization ---
        # Get the original orbital radii from the data
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()

        # Avoid division by zero if all radii are the same
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0

        # Define the range for the visual radii in the plot (e.g., from 2 to 8 units)
        min_visual_radius, max_visual_radius = 2, 8.0
        visual_range = max_visual_radius - min_visual_radius

        # Generate evenly spaced angles to position the planets around the star
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        # Loop through each connected "planet" to calculate its position
        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            # Scale the original radius to the new visual range
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)

            # Get the angle for this planet
            theta = thetas[i]

            # Calculate and store the (x, y, z) position using polar coordinates. Z is 0 for a flat system.
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    # Draw a dotted circle for each unique orbital radius
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100) # 100 points for a smooth circle
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(100, 100, 100, 0.5)', width=1, dash='dot'), # Faint, dotted line
            hoverinfo='none' # These rings should not be interactive
        ))

    # Create normalizers to map financial data values to the 0-1 range for the colormap
    norm_daily_change = mcolors.Normalize(vmin=-5, vmax=5) # For planet colors
    norm_gravitational_impact = mcolors.Normalize(vmin=-100, vmax=100) # For star color

    # List to hold annotations (text labels) for the scene
    scene_annotations = []

    # --- Loop Through Each Node (Star and Planets) to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        # Get financial info for the current node from the screener dataframe
        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue # Skip if no data is found
        screener_info = screener_info_row.iloc[0]

        # Format the market cap for readability (Billions or Trillions)
        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"


        # Define the range for the visual size of the spheres
        min_visual_size, max_visual_size = 0.3, 1.0

        # --- Configure Node Properties (Text, Color, Size) ---
        if is_source: # If the node is the central star
            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on gravitational impact
            node_color = mcolors.to_hex(cmap(norm_gravitational_impact(source_info['gravitational_impact'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))

        else: # If the node is a planet
            processed_info_row = ticker_connections[ticker_connections['target'] == node_name]
            if processed_info_row.empty: continue # Skip if no processed data
            processed_info = processed_info_row.iloc[0]

            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on daily price change
            node_color = mcolors.to_hex(cmap(norm_daily_change(processed_info['Daily Change'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))

        # --- Create and add the wireframe sphere for the current node ---
        sphere_x, sphere_y, sphere_z = create_wireframe_sphere(center_x, center_y, center_z, radius, node_color)
        fig.add_trace(go.Scatter3d(
            x=sphere_x, y=sphere_y, z=sphere_z,
            mode='lines',
            line=dict(color=node_color, width=2),
            hoverinfo='none' # The wireframe itself doesn't have a hover label
        ))

        # --- Dynamic Label Positioning using Scene Annotations ---
        # Add a text label for each node that is rendered on top of the 3D scene.
        scene_annotations.append(
            dict(
                x=center_x, y=center_y, z=center_z, # Position the label
                text=f"<b>{node_name}</b>",
                showarrow=False,
                # For planets, use a black font on a colored background. For the star, use white on a transparent background.
                font=dict(color='white' if not is_source else '#FFFFFF', size=16),
                # For planets, set the background color to the planet's color. For the star, it's transparent.
                bgcolor="rgba(0,0,0,0)"
            )
        )

        # --- HOVER INFO TRICK ---
        # Scene annotations don't support rich hover text. So, we add an INVISIBLE marker at the
        # center of each sphere. We make this marker large enough to be easily hovered over and
        # assign the detailed text to its `hovertext` property.
        fig.add_trace(go.Scatter3d(
            x=[center_x], y=[center_y], z=[center_z],
            mode='markers',
            marker=dict(size=radius*20, color='rgba(0,0,0,0)'), # Invisible, but sized for hovering
            hoverinfo='text',
            hovertext=text # This is the detailed text that will appear on hover
        ))

    # --- Configure Final Layout ---
    # Get the full company name for the title
    company_name_row = screener_data_df[screener_data_df['code'] == source_ticker]
    company_name = company_name_row['name'].iloc[0] if not company_name_row.empty else source_ticker # Use ticker if name not found
    fig.update_layout(
        title=f'Financial Solar System for {company_name} ({source_ticker})',
        title_x=0.5, # Center the title
        scene=dict(
            # Hide the 3D axes, grid, and background for a cleaner "space" look
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            # Set the camera's position (eye) and the point it looks at (center)
            camera=dict(
                eye=dict(x=0.97, y=0.97, z=0.5),      # Camera position is below the plane
                center=dict(x=0, y=0, z=-0.3)       # Look at a point slightly above the origin
            ),
            # Use 'data' aspect mode to ensure spheres look like spheres and not ellipsoids
            aspectmode='data',
            # Add the text annotations to the scene
            annotations=scene_annotations
        ),
        margin=dict(l=0, r=0, b=0, t=40), # Reduce margins to maximize plot area
        showlegend=False, # We don't need a legend for this plot
        paper_bgcolor='black', # Background of the entire figure area
        plot_bgcolor='black', # Background of the plotting area itself
        # Apply the font to the entire plot (title, etc.) and increase size
        font=dict(
            family="'VT323', monospace",
            color='#39FF14',
            size=18
        )
    )
    return fig

# --- Dash App Definition ---
# Initialize the Dash application
app = Dash(__name__)

# --- Styling Dictionaries for the UI ---
# Reusable styles for a consistent "terminal" look and feel using the VT323 font
terminal_font = {'fontFamily': "'VT323', monospace", 'color': '#39FF14', 'fontSize': '18px'}
terminal_container_style = {'backgroundColor': 'black', 'border': '1px solid #39FF14', 'padding': '10px 15px', 'margin': '10px', 'borderRadius': '5px'}
terminal_header_style = {'color': '#00FFFF', 'textAlign': 'center', 'text-decoration': 'underline', 'marginTop': 0, 'marginBottom': '10px','fontFamily': "'VT323', monospace", 'fontSize': '22px'}
button_style_green = {**terminal_font, 'backgroundColor': '#33CC33', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}
button_style_red = {**terminal_font, 'backgroundColor': '#FF0000', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}


# Assume screener_data_df is loaded and available in the global scope
# Create the list of options for the dropdown menu from the screener data
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
# Set a default ticker for the app to load initially
default_ticker = 'AAPL' if 'AAPL' in screener_data_df['code'].unique() else screener_data_df['code'].iloc[0]

# --- App Layout ---
# This defines the HTML structure of the web page.
app.layout = html.Div(style={'backgroundColor': 'black', 'color': '#39FF14', **terminal_font}, children=[
    # Import the Google Font
    html.Link(
        rel='stylesheet',
        href='https://fonts.googleapis.com/css2?family=VT323&display=swap'
    ),

    # Main title of the application
    html.H1("The Financial Planetarium", style={'textAlign': 'center', 'paddingTop': '20px', **terminal_font, 'fontSize': '36px', 'color': '#00FFFF'}),

    # Container for the dropdown menu
    html.Div([
        html.Label("Select a Stock:", style={'marginRight': '10px', **terminal_font}),
        dcc.Dropdown(
            id='ticker-dropdown', # Unique ID for this component
            options=ticker_options, # The list of tickers to choose from
            value=default_ticker, # The initial value
            style={'width': '300px', 'display': 'inline-block', **terminal_font, 'color': 'black'}
        ),
    ], style={'textAlign': 'center', 'padding': '20px'}),

    # Main content area with the graph and info tables, laid out horizontally
    html.Div([
        # --- Left Panel: Top Predictions ---
        html.Div([
            html.H3("Top Predictions", style=terminal_header_style),
            # Container for the two columns of buttons
            html.Div([
                # Column 1: Positive Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.2f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_green
                    ) for index, row in top_positive_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginRight': '5px'}
                ),
                # Column 2: Negative Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.1f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_red
                    ) for index, row in top_negative_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginLeft': '5px'}
                ),
            ], style={'display': 'flex', 'flexDirection': 'row'})
        ], style={**terminal_container_style, 'width': '20%', 'alignSelf': 'flex-start'}),

        # --- Center Panel: The Graph ---
        dcc.Graph(id='network-graph', style={'height': '50vh', 'width': '55%'}),

        # --- Right Panel: Info and Legend ---
        html.Div([
            html.Div(id='star-info-table', style=terminal_container_style),
            html.Div([
                html.H3("Legend", style=terminal_header_style),
                html.Div(style={
                    'height': '20px',
                    'width': '100%',
                    'background': f"linear-gradient(to right, {', '.join(colors)})",
                    'borderRadius': '3px'
                }),
                html.Div([
                    html.Span("Decrease", style={'textAlign': 'left', 'flex': 1}),
                    html.Span("Stable", style={'textAlign': 'center', 'flex': 1}),
                    html.Span("Increase", style={'textAlign': 'right', 'flex': 1})
                ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '5px'})
            ], style=terminal_container_style)
        ], style={'width': '25%', 'display': 'flex', 'flexDirection': 'column'})

    ], style={'display': 'flex', 'flexDirection': 'row'}),

    # A Div below the main content to hold the "Planet Information" table
    html.Div(id='planet-info-table', style={**terminal_container_style, 'marginTop': '20px'}),
])

# --- Callback for Prediction Buttons ---
@app.callback(
    Output('ticker-dropdown', 'value'),
    Input({'type': 'prediction-button', 'ticker': ALL}, 'n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_from_button(n_clicks):
    # Check if any button was actually clicked
    if not any(n_clicks):
        raise PreventUpdate

    # Get the ID of the button that was clicked
    ctx = callback_context
    button_id_str = ctx.triggered[0]['prop_id'].split('.')[0]
    button_id = json.loads(button_id_str)

    # Return the ticker from the button's ID to update the dropdown
    return button_id['ticker']

# --- Callback to Update All Components ---
@app.callback(
    [Output('network-graph', 'figure'),
     Output('star-info-table', 'children'),
     Output('planet-info-table', 'children')],
    [Input('ticker-dropdown', 'value')]
)
def update_all_components(selected_ticker):
    """
    This function is triggered whenever the user selects a new ticker from the dropdown.
    It re-calculates and re-generates all the visual components of the app.
    """
    # NOTE: The data processing part is assumed to happen here.
    # In a real application, you would load or calculate your DataFrames.
    # The following line is an example and is commented out because it depends
    # on variables not defined in this standalone script.

    global processed_data_df, source_data_df, screener_data_df, six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations,
        three_month_spearman_lagged_correlations,
        screener_data_df,
        selected_ticker,
        min_nodes,
        max_nodes,
        threshold_percent
    )


    # Check if selected_ticker exists in screener_data_df and source_data_df
    if selected_ticker not in screener_data_df['code'].unique() or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='black', font_color='#39FF14', font_family="'VT323', monospace")
        empty_star_table = [html.H3("Star Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        empty_planet_table = [html.H3("Planet Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        return empty_fig, empty_star_table, empty_planet_table


    # Generate the 3D plot using the main visualization function
    fig = solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df)

    # --- Generate Star Information Table ---
    # Fetch the data for the selected star
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    market_cap = star_info_screener.get('market_capitalization', 0)
    market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"

    # Build the HTML components for the star info box
    star_table_children = [
        html.H3("Star Information", style=terminal_header_style),
        html.P(f"Name: {star_info_screener['name']} ({selected_ticker})", style=terminal_font),
        html.P(f"Market Cap: {market_cap_str}", style=terminal_font),
        html.P(f"Gravitational Impact: {star_info_source['gravitational_impact']:.2f}%", style=terminal_font),
    ]

    # --- Generate Planet Information Table ---
    # Get the data for all planets orbiting the selected star
    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker]

    # --- Style definitions for the planet table ---
    # Style for the table header (solid green bar, black bold text, left-aligned)
    table_header_style = {
        **terminal_font, # Inherit base font styles
        'backgroundColor': '#39FF14',
        'color': 'black',
        'fontWeight': 'bold',
        'textAlign': 'left',
        'padding': '8px'
    }
    # Style for the table data cells (left-aligned text with padding)
    table_cell_style = {
        **terminal_font, # Inherit base font styles
        'textAlign': 'left',
        'padding': '8px',
        'borderBottom': '1px solid rgba(57, 255, 20, 0.3)' # Faint green line for row separation
    }

    # Define the table header using the new style
    table_header = [html.Thead(html.Tr([html.Th(col, style=table_header_style) for col in ["Ticker", "Name", "Market Cap", "Daily Change", "Correlation", "Gravitational Force"]]))]

    table_rows = []
    # Loop through the planet data to create a row for each one, applying the cell style
    for i, planet_row in planets_df.iterrows():
        screener_info_row = screener_data_df[screener_data_df['code'] == planet_row['target']]
        if screener_info_row.empty: continue # Skip if no screener data for planet

        screener_info = screener_info_row.iloc[0]
        planet_mcap_str = f"${screener_info['market_capitalization']/1e12:.2f}T" if screener_info['market_capitalization'] > 1e12 else f"${screener_info['market_capitalization']/1e9:.2f}B"
        table_rows.append(html.Tr([
            html.Td(planet_row['target'], style=table_cell_style),
            html.Td(screener_info['name'], style=table_cell_style),
            html.Td(planet_mcap_str, style=table_cell_style),
            html.Td(f"{planet_row['Daily Change']:.2f}%", style=table_cell_style),
            html.Td(f"{planet_row['unified_correlation']:.3f}", style=table_cell_style),
            html.Td(f"{planet_row['signed_gravitational_force']:.2f}", style=table_cell_style),
        ]))

    # Combine the header and rows into a full table
    planet_table_children = [
        html.H3("Planet Information", style=terminal_header_style),
        html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse', 'borderSpacing': 0})
    ]

    # Return the generated figure and table contents to the appropriate 'Output' components
    return fig, star_table_children, planet_table_children

# --- Run the App ---
# This line is standard for running a Python script directly.
# The `if __name__ == '__main__':` block ensures this code only runs
# when the script is executed, not when it's imported as a module.
if __name__ == '__main__':
    # Starts the web server. `debug=True` allows for hot-reloading and provides error messages in the browser.
    app.run(debug=True, port=8051)


<IPython.core.display.Javascript object>

In [ ]:
aaaa

In [47]:
# The Financial Planetarium
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import numpy as np   # For numerical operations, especially for generating coordinates
import plotly.graph_objects as go  # The main library for creating interactive plots
import matplotlib.colors as mcolors # For creating custom color maps and normalizing color scales
from dash import Dash, dcc, html, callback_context   # The framework for building the web application interface
from dash.dependencies import Input, Output, State, ALL # To create interactivity in the Dash app (callbacks)
from dash.exceptions import PreventUpdate
import json


# --- Helper Function to Create Wireframe Spheres ---
def create_wireframe_sphere(center_x, center_y, center_z, radius, color, n_lat=20, n_lon=20):
    """
    Generates the coordinate data for a 3D wireframe sphere for a Plotly graph.
    This function calculates points along latitude and longitude lines to give the
    appearance of a sphere without rendering a solid surface.

    Args:
        center_x, center_y, center_z (float): The center coordinates of the sphere.
        radius (float): The radius of the sphere.
        color (str): The color of the wireframe lines (in a format Plotly understands, e.g., hex code).
        n_lat (int): The number of latitude lines to draw. More lines create a denser sphere.
        n_lon (int): The number of longitude lines to draw. More lines create a denser sphere.

    Returns:
        tuple: A tuple containing three lists (x_lines, y_lines, z_lines) of coordinates
               for the sphere's lines. `None` values are used to separate the lines.
    """
    # Initialize lists to hold the coordinates for all line segments
    x_lines, y_lines, z_lines = [], [], []

    # --- Generate latitude lines (horizontal rings) ---
    # Loop through each latitude angle from the North Pole to the South Pole
    for i in range(n_lat + 1):
        # Calculate the latitude angle. np.pi/2 is the North Pole, -np.pi/2 is the South Pole.
        lat_angle = np.pi / 2 - (i * np.pi / n_lat)
        z = radius * np.sin(lat_angle) # Z coordinate is constant for a given latitude
        r = radius * np.cos(lat_angle) # The radius of the horizontal ring at this latitude

        # Loop through each longitude to draw the points on the ring
        for j in range(n_lon + 1):
            lon_angle = j * 2 * np.pi / n_lon # Go a full circle (2*pi)
            x = r * np.cos(lon_angle) # Calculate x based on the ring radius and longitude
            y = r * np.sin(lon_angle) # Calculate y based on the ring radius and longitude

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full latitude line. This tells Plotly to stop drawing
        # the current line and start a new one, preventing lines from connecting across the sphere.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    # --- Generate longitude lines (vertical meridians) ---
    # Loop through each longitude angle
    for i in range(n_lon + 1):
        lon_angle = i * 2 * np.pi / n_lon # The angle for this specific meridian

        # Loop through each latitude to draw the points along the vertical line
        for j in range(n_lat + 1):
            lat_angle = np.pi / 2 - (j * np.pi / n_lat) # Go from North to South Pole
            z = radius * np.sin(lat_angle) # Calculate z for this point on the meridian
            r = radius * np.cos(lat_angle) # Calculate the horizontal distance from the z-axis

            # Calculate x and y based on the constant longitude angle
            x = r * np.cos(lon_angle)
            y = r * np.sin(lon_angle)

            # Append the calculated point, shifted by the sphere's center coordinates
            x_lines.append(x + center_x)
            y_lines.append(y + center_y)
            z_lines.append(z + center_z)

        # Add `None` after each full longitude line to start a new line.
        x_lines.append(None)
        y_lines.append(None)
        z_lines.append(None)

    return x_lines, y_lines, z_lines

# --- Define Color Mapping (Global for use in layout and plotting) ---
colors = ['#FF0000', '#F04435', '#E47C61', '#D9B28B', '#A5BA6F', '#73C255', '#33CC33']
cmap = mcolors.LinearSegmentedColormap.from_list('my_cmap', colors)

# --- Main Plotting Function ---
def solar_system_visual(source_ticker, processed_data_df, source_data_df, screener_data_df):
    """
    Creates the main 3D solar system visualization for a given central stock (source_ticker).

    Args:
        source_ticker (str): The ticker symbol for the central star of the system.
        processed_data_df (pd.DataFrame): DataFrame with data about connections (planets).
        source_data_df (pd.DataFrame): DataFrame with data about the source stocks (stars).
        screener_data_df (pd.DataFrame): DataFrame with general financial info for all tickers.

    Returns:
        go.Figure: A Plotly Figure object containing the complete 3D visualization.
    """
    # Filter the data to get only the connections for the selected source_ticker
    ticker_connections = processed_data_df[processed_data_df['source'] == source_ticker].copy()

    # Get the specific row of data for the source_ticker itself
    source_info_row = source_data_df[source_data_df['ticker'] == source_ticker]

    # If there's no data for the selected ticker, return an empty figure with a message
    if ticker_connections.empty or source_info_row.empty:
        return go.Figure().update_layout(title=f"Data not available for {source_ticker}", title_x=0.5, paper_bgcolor='black', font_color='white')

    # Extract the source info into a pandas Series for easier access
    source_info = source_info_row.iloc[0]

    # Initialize the Plotly Figure
    fig = go.Figure()

    # Create a dictionary to store the 3D positions (x, y, z) of each stock (node).
    # The source ticker (the "star") is always placed at the origin (0, 0, 0).
    pos = {source_ticker: (0, 0, 0)}

    # Filter out any connections where the target is the source itself
    actual_target_connections = ticker_connections[ticker_connections['target'] != source_ticker].copy()
    num_connections = len(actual_target_connections)

    # List to store the calculated visual radii for the orbital rings
    radii_for_rings = []

    # Calculate positions for the "planets" (connected tickers) if any exist
    if num_connections > 0:
        # --- Scale orbital distances for better visualization ---
        # Get the original orbital radii from the data
        original_radii = actual_target_connections['Orbital Radius']
        min_rad, max_rad = original_radii.min(), original_radii.max()

        # Avoid division by zero if all radii are the same
        rad_range = max_rad - min_rad if max_rad > min_rad else 1.0

        # Define the range for the visual radii in the plot (e.g., from 2 to 8 units)
        min_visual_radius, max_visual_radius = 2, 8.0
        visual_range = max_visual_radius - min_visual_radius

        # Generate evenly spaced angles to position the planets around the star
        thetas = np.linspace(0, 2 * np.pi, num_connections, endpoint=False)

        # Loop through each connected "planet" to calculate its position
        for i, (index, row) in enumerate(actual_target_connections.iterrows()):
            # Scale the original radius to the new visual range
            scaled_radius = ((row['Orbital Radius'] - min_rad) / rad_range) * visual_range + min_visual_radius
            radii_for_rings.append(scaled_radius)

            # Get the angle for this planet
            theta = thetas[i]

            # Calculate and store the (x, y, z) position using polar coordinates. Z is 0 for a flat system.
            pos[row['target']] = (scaled_radius * np.cos(theta), scaled_radius * np.sin(theta), 0)

    # --- Add Orbital Rings ---
    # Draw a dotted circle for each unique orbital radius
    for r in sorted(list(set(radii_for_rings))):
        theta_ring = np.linspace(0, 2 * np.pi, 100) # 100 points for a smooth circle
        fig.add_trace(go.Scatter3d(
            x=r * np.cos(theta_ring), y=r * np.sin(theta_ring), z=np.zeros(100),
            mode='lines',
            line=dict(color='rgba(100, 100, 100, 0.5)', width=1, dash='dot'), # Faint, dotted line
            hoverinfo='none' # These rings should not be interactive
        ))

    # Create normalizers to map financial data values to the 0-1 range for the colormap
    norm_daily_change = mcolors.Normalize(vmin=-5, vmax=5) # For planet colors
    norm_gravitational_impact = mcolors.Normalize(vmin=-100, vmax=100) # For star color

    # List to hold annotations (text labels) for the scene
    scene_annotations = []

    # --- Loop Through Each Node (Star and Planets) to Draw Them ---
    for node_name, coords in pos.items():
        center_x, center_y, center_z = coords
        is_source = (node_name == source_ticker)

        # Get financial info for the current node from the screener dataframe
        screener_info_row = screener_data_df[screener_data_df['code'] == node_name]
        if screener_info_row.empty: continue # Skip if no data is found
        screener_info = screener_info_row.iloc[0]

        # Format the market cap for readability (Billions or Trillions)
        market_cap = screener_info.get('market_capitalization', 0)
        market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"


        # Define the range for the visual size of the spheres
        min_visual_size, max_visual_size = 0.3, 1.0

        # --- Configure Node Properties (Text, Color, Size) ---
        if is_source: # If the node is the central star
            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on gravitational impact
            node_color = mcolors.to_hex(cmap(norm_gravitational_impact(source_info['gravitational_impact'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (source_info['source_planet_radius'] * (max_visual_size - min_visual_size))

        else: # If the node is a planet
            processed_info_row = ticker_connections[ticker_connections['target'] == node_name]
            if processed_info_row.empty: continue # Skip if no processed data
            processed_info = processed_info_row.iloc[0]

            text = (f"<b>{screener_info.get('name', node_name)} ({node_name})</b><br>"
                    f"Industry: {screener_info.get('industry', 'N/A')}<br>"
                    f"Sector: {screener_info.get('sector', 'N/A')}<br>"
                    f"Avg Volume (1d): {screener_info.get('avgvol_1d', 'N/A')}<br>"
                    f"Daily Change: {processed_info['Daily Change']:.2f}%<br>"
                    f"Market Cap: {market_cap_str}")
            # Color is based on daily price change
            node_color = mcolors.to_hex(cmap(norm_daily_change(processed_info['Daily Change'])))
            # Radius is based on its own calculated radius property, scaled to the visual size range
            radius = min_visual_size + (processed_info['Planet Radius'] * (max_visual_size - min_visual_size))

        # --- Create and add the wireframe sphere for the current node ---
        sphere_x, sphere_y, sphere_z = create_wireframe_sphere(center_x, center_y, center_z, radius, node_color)
        fig.add_trace(go.Scatter3d(
            x=sphere_x, y=sphere_y, z=sphere_z,
            mode='lines',
            line=dict(color=node_color, width=2),
            hoverinfo='none' # The wireframe itself doesn't have a hover label
        ))

        # --- Dynamic Label Positioning using Scene Annotations ---
        # Add a text label for each node that is rendered on top of the 3D scene.
        scene_annotations.append(
            dict(
                x=center_x, y=center_y, z=center_z, # Position the label
                text=f"<b>{node_name}</b>",
                showarrow=False,
                # For planets, use a black font on a colored background. For the star, use white on a transparent background.
                font=dict(color='white' if not is_source else '#FFFFFF', size=16),
                # For planets, set the background color to the planet's color. For the star, it's transparent.
                bgcolor="rgba(0,0,0,0)"
            )
        )

        # --- HOVER INFO TRICK ---
        # Scene annotations don't support rich hover text. So, we add an INVISIBLE marker at the
        # center of each sphere. We make this marker large enough to be easily hovered over and
        # assign the detailed text to its `hovertext` property.
        fig.add_trace(go.Scatter3d(
            x=[center_x], y=[center_y], z=[center_z],
            mode='markers',
            marker=dict(size=radius*20, color='rgba(0,0,0,0)'), # Invisible, but sized for hovering
            hoverinfo='text',
            hovertext=text # This is the detailed text that will appear on hover
        ))

    # --- Configure Final Layout ---
    # Get the full company name for the title
    company_name_row = screener_data_df[screener_data_df['code'] == source_ticker]
    company_name = company_name_row['name'].iloc[0] if not company_name_row.empty else source_ticker # Use ticker if name not found
    fig.update_layout(
        title=f'Financial Solar System for {company_name} ({source_ticker})',
        title_x=0.5, # Center the title
        scene=dict(
            # Hide the 3D axes, grid, and background for a cleaner "space" look
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            # Set the camera's position (eye) and the point it looks at (center)
            camera=dict(
                eye=dict(x=0.97, y=0.97, z=0.5),      # Camera position is below the plane
                center=dict(x=0, y=0, z=-0.3)       # Look at a point slightly above the origin
            ),
            # Use 'data' aspect mode to ensure spheres look like spheres and not ellipsoids
            aspectmode='data',
            # Add the text annotations to the scene
            annotations=scene_annotations
        ),
        margin=dict(l=0, r=0, b=0, t=40), # Reduce margins to maximize plot area
        showlegend=False, # We don't need a legend for this plot
        paper_bgcolor='black', # Background of the entire figure area
        plot_bgcolor='black', # Background of the plotting area itself
        # Apply the font to the entire plot (title, etc.) and increase size
        font=dict(
            family="'VT323', monospace",
            color='#39FF14',
            size=18
        )
    )
    return fig

# --- Dash App Definition ---
# Initialize the Dash application
app = Dash(__name__)

# --- Styling Dictionaries for the UI ---
# Reusable styles for a consistent "terminal" look and feel using the VT323 font
terminal_font = {'fontFamily': "'VT323', monospace", 'color': '#39FF14', 'fontSize': '18px'}
terminal_container_style = {'backgroundColor': 'black', 'border': '1px solid #39FF14', 'padding': '10px 15px', 'margin': '10px', 'borderRadius': '5px'}
terminal_header_style = {'color': '#00FFFF', 'textAlign': 'center', 'text-decoration': 'underline', 'marginTop': 0, 'marginBottom': '10px','fontFamily': "'VT323', monospace", 'fontSize': '22px'}
button_style_green = {**terminal_font, 'backgroundColor': '#33CC33', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}
button_style_red = {**terminal_font, 'backgroundColor': '#FF0000', 'color': 'black', 'fontWeight': 'bold', 'border': 'none', 'padding': '5px 10px', 'margin': '3px 0', 'borderRadius': '3px', 'textAlign': 'center', 'width': '100%', 'fontSize': '16px'}


# Assume screener_data_df is loaded and available in the global scope
# Create the list of options for the dropdown menu from the screener data
ticker_options = [{'label': row['name'] + f" ({row['code']})", 'value': row['code']} for index, row in screener_data_df.iterrows()]
# Set a default ticker for the app to load initially
default_ticker = 'AAPL' if 'AAPL' in screener_data_df['code'].unique() else screener_data_df['code'].iloc[0]

# --- App Layout ---
# This defines the HTML structure of the web page.
app.layout = html.Div(style={'backgroundColor': 'black', 'color': '#39FF14', **terminal_font}, children=[
    # Import the Google Font
    html.Link(
        rel='stylesheet',
        href='https://fonts.googleapis.com/css2?family=VT323&display=swap'
    ),

    # Main title of the application
    html.H1("The Financial Planetarium", style={'textAlign': 'center', 'paddingTop': '20px', **terminal_font, 'fontSize': '36px', 'color': '#00FFFF'}),

    # Container for the dropdown menu
    html.Div([
        html.Label("Select a Stock:", style={'marginRight': '10px', **terminal_font}),
        dcc.Dropdown(
            id='ticker-dropdown', # Unique ID for this component
            options=ticker_options, # The list of tickers to choose from
            value=default_ticker, # The initial value
            style={'width': '300px', 'display': 'inline-block', **terminal_font, 'color': 'black'}
        ),
    ], style={'textAlign': 'center', 'padding': '20px'}),

    # Main content area with the graph and info tables, laid out horizontally
    html.Div([
        # --- Left Panel: Top Predictions ---
        html.Div([
            html.H3("Top Predictions", style=terminal_header_style),
            # Container for the two columns of buttons
            html.Div([
                # Column 1: Positive Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.2f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_green
                    ) for index, row in top_positive_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginRight': '5px'}
                ),
                # Column 2: Negative Buttons
                html.Div(
                    [html.Button(
                        f"{row['ticker']}: {row['gravitational_impact']:.1f}%",
                        id={'type': 'prediction-button', 'ticker': row['ticker']},
                        style=button_style_red
                    ) for index, row in top_negative_impacts.head(6).iterrows()],
                    style={'flex': 1, 'marginLeft': '5px'}
                ),
            ], style={'display': 'flex', 'flexDirection': 'row'})
        ], style={**terminal_container_style, 'width': '20%', 'alignSelf': 'flex-start'}),

        # --- Center Panel: The Graph ---
        dcc.Graph(id='network-graph', style={'height': '50vh', 'width': '55%'}),

        # --- Right Panel: Info and Legend ---
        html.Div([
            html.Div(id='star-info-table', style=terminal_container_style),
            html.Div([
                html.H3("Legend", style=terminal_header_style),
                html.Div(style={
                    'height': '20px',
                    'width': '100%',
                    'background': f"linear-gradient(to right, {', '.join(colors)})",
                    'borderRadius': '3px'
                }),
                html.Div([
                    html.Span("Decrease", style={'textAlign': 'left', 'flex': 1}),
                    html.Span("Stable", style={'textAlign': 'center', 'flex': 1}),
                    html.Span("Increase", style={'textAlign': 'right', 'flex': 1})
                ], style={'display': 'flex', 'justifyContent': 'space-between', 'marginTop': '5px'})
            ], style=terminal_container_style)
        ], style={'width': '25%', 'display': 'flex', 'flexDirection': 'column'})

    ], style={'display': 'flex', 'flexDirection': 'row'}),

    # A Div below the main content to hold the "Planet Information" table
    html.Div(id='planet-info-table', style={**terminal_container_style, 'marginTop': '20px'}),
])

# --- Callback for Prediction Buttons ---
@app.callback(
    Output('ticker-dropdown', 'value'),
    Input({'type': 'prediction-button', 'ticker': ALL}, 'n_clicks'),
    prevent_initial_call=True
)
def update_dropdown_from_button(n_clicks):
    # Check if any button was actually clicked
    if not any(n_clicks):
        raise PreventUpdate

    # Get the ID of the button that was clicked
    ctx = callback_context
    button_id_str = ctx.triggered[0]['prop_id'].split('.')[0]
    button_id = json.loads(button_id_str)

    # Return the ticker from the button's ID to update the dropdown
    return button_id['ticker']

# --- Callback to Update All Components ---
@app.callback(
    [Output('network-graph', 'figure'),
     Output('star-info-table', 'children'),
     Output('planet-info-table', 'children')],
    [Input('ticker-dropdown', 'value')]
)
def update_all_components(selected_ticker):
    """
    This function is triggered whenever the user selects a new ticker from the dropdown.
    It re-calculates and re-generates all the visual components of the app.
    """
    # NOTE: The data processing part is assumed to happen here.
    # In a real application, you would load or calculate your DataFrames.
    # The following line is an example and is commented out because it depends
    # on variables not defined in this standalone script.

    global processed_data_df, source_data_df, screener_data_df, six_month_spearman_lagged_correlations, three_month_spearman_lagged_correlations
    processed_data_df, source_data_df = process_and_score_stocks(
        six_month_spearman_lagged_correlations,
        three_month_spearman_lagged_correlations,
        screener_data_df,
        selected_ticker,
        min_nodes,
        max_nodes,
        threshold_percent
    )


    # Check if selected_ticker exists in screener_data_df and source_data_df
    if selected_ticker not in screener_data_df['code'].unique() or source_data_df.empty:
        empty_fig = go.Figure().update_layout(title=f"Data not available for {selected_ticker}", title_x=0.5, paper_bgcolor='black', font_color='#39FF14', font_family="'VT323', monospace")
        empty_star_table = [html.H3("Star Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        empty_planet_table = [html.H3("Planet Information", style=terminal_header_style), html.P(f"Data not available for {selected_ticker}", style=terminal_font)]
        return empty_fig, empty_star_table, empty_planet_table


    # Generate the 3D plot using the main visualization function
    fig = solar_system_visual(selected_ticker, processed_data_df, source_data_df, screener_data_df)

    # --- Generate Star Information Table ---
    # Fetch the data for the selected star
    star_info_screener = screener_data_df[screener_data_df['code'] == selected_ticker].iloc[0]
    star_info_source = source_data_df[source_data_df['ticker'] == selected_ticker].iloc[0]
    market_cap = star_info_screener.get('market_capitalization', 0)
    market_cap_str = f"${market_cap/1e12:.2f}T" if market_cap > 1e12 else f"${market_cap/1e9:.2f}B"

    # Build the HTML components for the star info box
    star_table_children = [
        html.H3("Star Information", style=terminal_header_style),
        html.P(f"Name: {star_info_screener['name']} ({selected_ticker})", style=terminal_font),
        html.P(f"Market Cap: {market_cap_str}", style=terminal_font),
        html.P(f"Gravitational Impact: {star_info_source['gravitational_impact']:.2f}%", style=terminal_font),
    ]

    # --- Generate Planet Information Table ---
    # Get the data for all planets orbiting the selected star
    planets_df = processed_data_df[processed_data_df['source'] == selected_ticker]

    # --- Style definitions for the planet table ---
    # Style for the table header (solid green bar, black bold text, left-aligned)
    table_header_style = {
        **terminal_font, # Inherit base font styles
        'backgroundColor': '#39FF14',
        'color': 'black',
        'fontWeight': 'bold',
        'textAlign': 'left',
        'padding': '8px'
    }
    # Style for the table data cells (left-aligned text with padding)
    table_cell_style = {
        **terminal_font, # Inherit base font styles
        'textAlign': 'left',
        'padding': '8px',
        'borderBottom': '1px solid rgba(57, 255, 20, 0.3)' # Faint green line for row separation
    }

    # Define the table header using the new style
    table_header = [html.Thead(html.Tr([html.Th(col, style=table_header_style) for col in ["Ticker", "Name", "Market Cap", "Daily Change", "Correlation", "Gravitational Force"]]))]

    table_rows = []
    # Loop through the planet data to create a row for each one, applying the cell style
    for i, planet_row in planets_df.iterrows():
        screener_info_row = screener_data_df[screener_data_df['code'] == planet_row['target']]
        if screener_info_row.empty: continue # Skip if no screener data for planet

        screener_info = screener_info_row.iloc[0]
        planet_mcap_str = f"${screener_info['market_capitalization']/1e12:.2f}T" if screener_info['market_capitalization'] > 1e12 else f"${screener_info['market_capitalization']/1e9:.2f}B"
        table_rows.append(html.Tr([
            html.Td(planet_row['target'], style=table_cell_style),
            html.Td(screener_info['name'], style=table_cell_style),
            html.Td(planet_mcap_str, style=table_cell_style),
            html.Td(f"{planet_row['Daily Change']:.2f}%", style=table_cell_style),
            html.Td(f"{planet_row['unified_correlation']:.3f}", style=table_cell_style),
            html.Td(f"{planet_row['signed_gravitational_force']:.2f}", style=table_cell_style),
        ]))

    # Combine the header and rows into a full table
    planet_table_children = [
        html.H3("Planet Information", style=terminal_header_style),
        html.Table(table_header + [html.Tbody(table_rows)], style={'width': '100%', 'borderCollapse': 'collapse', 'borderSpacing': 0})
    ]

    # Return the generated figure and table contents to the appropriate 'Output' components
    return fig, star_table_children, planet_table_children

# --- Run the App ---
# This line is standard for running a Python script directly.
# The `if __name__ == '__main__':` block ensures this code only runs
# when the script is executed, not when it's imported as a module.
if __name__ == '__main__':
    # Starts the web server. `debug=True` allows for hot-reloading and provides error messages in the browser.
    app.run(debug=True, port=8051)


<IPython.core.display.Javascript object>